Some investigation of the system and how the parametesr influence the solutions.

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [ ]:
def solve_AllenCahn(u0=None, L=1.0, N=200, dt=0.1, steps=500, epsilon=0.1, mu=1.0, rng=None, boundary_condition='Neumann'):
    if u0 is None:
        if rng is None:
            rng = np.random.default_rng()
        u = rng.uniform(-1, 1, N) / 1000  # Random initial condition
    else:
        u = u0

    dx = L / N        # Spatial step size
    x = np.linspace(0, L, N, endpoint=False)

    # Construct Laplacian with periodic boundary conditions
    main_diag = -2.0 * np.ones(N)
    off_diag = np.ones(N - 1)
    laplacian = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], shape=(N, N)).toarray()

    if boundary_condition in ['Neumann', 'neumann']:
        laplacian[0, 1] = 2.0   # Neumann BC at left boundary
        laplacian[-1, -2] = 2.0 # Neumann BC at right boundary
    elif boundary_condition in ['periodic', 'Periodic']:
        laplacian[0, -1] = laplacian[-1, 0] = 1.0
    else:
        raise NotImplementedError("Unknown boundary condition", boundary_condition)

    laplacian = laplacian / dx**2

    # Convert to sparse matrix for efficiency
    L_eps = epsilon**2 * laplacian
    I = np.eye(N)
    A = I - dt * L_eps  # Implicit matrix for diffusion

    # Time integration loop
    snapshots = np.empty((steps+1, N))
    snapshots[0] = u
    for n in range(steps):
        f_u = (u**3 - mu*u)  # Nonlinear term
        rhs = u - dt * f_u
        u = spsolve(A, rhs)
        # if n % 100 == 0:
        if True:
            snapshots[n+1] = u

    return snapshots

# Show one example
## Neumann BCs

In [ ]:
sol = solve_AllenCahn(epsilon=0.01)

plt.imshow(np.array(sol).T, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
plt.xlabel('Time step')
plt.ylabel('Spatial index')
plt.colorbar(label='u(x,t)')


## Periodic BCs

In [ ]:
sol = solve_AllenCahn(boundary_condition='periodic', epsilon=0.01)

plt.imshow(np.array(sol).T, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
plt.xlabel('Time step')
plt.ylabel('Spatial index')
plt.colorbar(label='u(x,t)')


# Solution grid, varying $\epsilon$ and $\mu$ (periodic BCs)

In [ ]:
rng = np.random.default_rng(43)  # For reproducibility

fig, axes = plt.subplots(6, 5, figsize=(15, 15))

# Parameters
epsilon_arr = 10**np.linspace(-3, -0.5, 6)
mu_arr = np.linspace(-0.1, 1.0, 5)

for i, epsilon in enumerate(epsilon_arr):
    for j, mu in enumerate(mu_arr):
        print(f'Solving for epsilon = {epsilon}, mu = {mu}')

        # Solve the Allen-Cahn equation
        sol = solve_AllenCahn(rng=rng, epsilon=epsilon, mu=mu, steps=1000, dt=0.1, boundary_condition='periodic')

        axes[i][j].imshow(sol.T, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
        axes[i][j].set_title(f'$\\epsilon={epsilon:.2}$, $\\mu={mu:.2}$')
        axes[i][j].set_xlabel('Time')
        axes[i][j].set_ylabel('Space')

plt.tight_layout()
plt.show()

# Vary $\epsilon$ (periodic BCs)

In [ ]:
rng = np.random.default_rng(42)  # For reproducibility

# Parameters
steps = 1000
N = 200
epsilon_arr = np.repeat(10**np.linspace(-3, -1, 513), 1)
# epsilon_arr = np.linspace(0.01, 0.3, 100)

solutions = np.empty((len(epsilon_arr), steps+1, N), dtype=float)

for i, epsilon in enumerate(epsilon_arr):
    print(f'Solving for epsilon = {epsilon}')

    # Solve the Allen-Cahn equation
    sol = solve_AllenCahn(rng=rng, epsilon=epsilon, steps=steps, dt=0.1, N=N, boundary_condition='periodic')

    solutions[i] = sol

## Plot solution norm vs $\epsilon$

In [ ]:
# Solution norm
solution_norms = np.linalg.norm(solutions[:, -1], axis=(-1))
plt.scatter(epsilon_arr, solution_norms, s=3, alpha=0.5)
plt.xlabel('$\epsilon$')
plt.ylabel('Solution norm')
plt.xscale('log')

In [ ]:
import pickle
with open('AllenCahn_periodic_varyEps.pkl', 'wb') as f:
    pickle.dump({'epsilon_arr': epsilon_arr, 'solutions': solutions}, f)

### Fit with multiple lines

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('AllenCahn_periodic_varyEps.pkl', 'rb') as f:
    data = pickle.load(f)

    print(data.keys())
    solutions = data['solutions']
    epsilon_arr = data['epsilon_arr']

# Solution norm
solution_norms = np.linalg.norm(solutions[:, -1], axis=(-1))
plt.scatter(epsilon_arr, solution_norms, s=3, alpha=0.5)
plt.xlabel('$\epsilon$')
plt.ylabel('Solution norm')
# plt.xscale('log')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RANSACRegressor, LinearRegression

def fit_multiple_lines(X, min_inliers=30, residual_threshold=0.05, max_lines=None):
    """
    Fit multiple straight lines using iterative RANSAC.

    Parameters
    ----------
    X : array, shape (n_samples, 2)
        Input points (x,y).
    min_inliers : int
        Stop if fewer than this many inliers found for a line.
    residual_threshold : float
        Maximum residual (distance) for a data point to be classified as an inlier.
    max_lines : int or None
        Maximum number of lines to fit (None = keep going until no inliers left).
    """
    lines = []
    remaining_idx = np.arange(X.shape[0])

    while remaining_idx.size >= min_inliers and (max_lines is None or len(lines) < max_lines):
        pts = X[remaining_idx]
        x = pts[:, 0].reshape(-1, 1)
        y = pts[:, 1]

        ransac = RANSACRegressor(
            estimator=LinearRegression(positive=True, fit_intercept=False),
            residual_threshold=residual_threshold,
            random_state=0
        )
        ransac.fit(x, y)

        inlier_mask = ransac.inlier_mask_
        if inlier_mask.sum() < min_inliers:
            break

        # Extract inliers
        inlier_idx = remaining_idx[inlier_mask]
        inlier_pts = X[inlier_idx]

        # Order points along the line (by x, since line is linear in x)
        ordered_idx = inlier_idx[np.argsort(inlier_pts[:, 0])]
        ordered_pts = X[ordered_idx]

        # Save line info
        lines.append({
            "coef": ransac.estimator_.coef_[0],
            "intercept": ransac.estimator_.intercept_,
            "inlier_indices": ordered_idx,
            "ordered_points": ordered_pts,
        })

        # Remove inliers from pool
        remaining_idx = remaining_idx[~inlier_mask]

    return lines, remaining_idx

# Test case
rng = np.random.default_rng(42)

# Line 1: y = 0.5x + 1
x1 = np.linspace(0, 10, 100)
y1 = 0.5 * x1 + 1 + rng.normal(0, 0.02, size=x1.size)

# Line 2: vertical-ish (x ~ 5) -> we fake it as steep slope
y2 = np.linspace(-2, 8, 80)
x2 = 5 + rng.normal(0, 0.05, size=y2.size)

# Line 3: y = -0.8x + 6
x3 = np.linspace(-2, 6, 120)
y3 = -0.8 * x3 + 6 + rng.normal(0, 0.02, size=x3.size)

# Mix all and add some outliers
X = np.vstack([np.c_[x1, y1], np.c_[x2, y2], np.c_[x3, y3],
                rng.uniform([-2, -2], [10, 8], size=(50, 2))])

lines, leftovers = fit_multiple_lines(X, min_inliers=40, residual_threshold=0.1)

print(f"Detected {len(lines)} lines; leftover points: {leftovers.size}")

# --- Plot ---
plt.scatter(X[:, 0], X[:, 1], s=10, c="lightgray", label="data")
colors = ["r", "g", "b", "m", "c"]
for k, L in enumerate(lines):
    pts = L["ordered_points"]
    plt.plot(pts[:, 0], pts[:, 1], colors[k % len(colors)] + "-", lw=2, label=f"Line {k+1}")
plt.legend()
plt.show()


In [ ]:
%matplotlib qt
X = np.stack((epsilon_arr, -solution_norms+np.max(solution_norms)), axis=1)
lines, leftovers = fit_multiple_lines(X, min_inliers=20, residual_threshold=0.05)

print(f"Detected {len(lines)} lines; leftover points: {leftovers.size}")

# --- Plot ---
# plt.scatter(X[:, 0], X[:, 1], s=10, c="lightgray", label="data")
plt.scatter(epsilon_arr, solution_norms, s=10, c="lightgray", label="data")
colors = ["r", "g", "b", "m", "c"]
for k, L in enumerate(lines):
    plt.axline(xy1=[0,np.max(solution_norms)], slope=-L["coef"], color=colors[k % len(colors)])
    # pts = L["ordered_points"]
    # plt.plot(pts[:, 0], pts[:, 1], colors[k % len(colors)] + "-", lw=2, label=f"Line {k+1}")
plt.legend()
plt.show()

## Plot boundary value vs $\epsilon$

In [ ]:
# (useless for periodic BCs)

# # Solution boundary value
# plt.figure(figsize=(4,3), dpi=200)
# sol_boundary_val = solutions[:, -1, 0]
# plt.scatter(epsilon_arr, sol_boundary_val, s=1, alpha=0.6)
# plt.xlabel('$\epsilon$')
# plt.ylabel('u(0)')
# plt.title('1D Allen-Cahn bifurcation diagram, end state')
# # plt.xscale('log')
# plt.grid(True)
# plt.show()

## Plot max. diff with neighbor vs $\epsilon$

In [ ]:
diff = np.diff(solutions[:, -1], append=solutions[:, -1, [0]], axis=1)
avg_diff = np.mean(diff, axis=-1)
max_diff = np.max(diff, axis=-1)
# plt.plot(epsilon_arr, avg_diff)
plt.scatter(epsilon_arr, max_diff, alpha=0.2, s=3)
plt.xlabel('$\epsilon$')
plt.ylabel('Maximum difference with neighbor')


## Plot nr of blobs vs $\epsilon$

In [ ]:
# to do: improve blob counting - use method from AllenCahn_FM.py

In [ ]:
rounded = (solutions[:, -1] > 0).astype(int)
diff = np.diff(rounded, axis=1)
nr_of_blobs = np.sum(np.abs(diff), axis=-1)  # Count the number of changes
plt.scatter(epsilon_arr, nr_of_blobs, alpha=0.2, s=3)
plt.xlabel('$\epsilon$')
plt.ylabel('Number of blobs')
plt.xscale('log')

## Plot all final states

In [ ]:
solutions.shape

In [ ]:
sol_temp = solutions[:, -1]
n_sol = len(sol_temp)
for i in range(n_sol//2):
    plt.plot(sol_temp[i], color=plt.cm.viridis(i / (len(solutions)-1)), alpha=0.5)
    if i % 10 == 0 and not i == 0:
        plt.show()
        plt.figure()

# Vary $\mu$  (periodic BCs)

In [ ]:
rng = np.random.default_rng(42)  # For reproducibility

# Parameters
steps = 1000
N = 200
mu_arr = np.linspace(-0.1, 1.0, 500)
solutions = np.empty((len(mu_arr), steps+1, N), dtype=float)

for i, mu in enumerate(mu_arr):
    print(f'Solving for mu = {mu}')

    # Solve the Allen-Cahn equation
    sol = solve_AllenCahn(rng=rng, mu=mu, steps=steps, dt=0.1, N=N, boundary_condition='periodic')

    solutions[i] = sol

## Plot solution norm vs $\mu$

In [ ]:
# Solution norm
solution_norms = np.linalg.norm(solutions[:, -1], axis=(-1))
plt.scatter(mu_arr, solution_norms, s=3)
plt.xlabel('$\mu$')
plt.ylabel('Solution norm')

## Plot average value of solution vs $\mu$

In [ ]:
# Avg value of solution
avg_value = np.mean(solutions[:, -1], axis=(-1))
plt.scatter(mu_arr, avg_value, s=3)
plt.xlabel('$\mu$')
plt.ylabel('Average solution value')

In [ ]:
import pickle
with open('AllenCahn_periodic_varyMu.pkl', 'wb') as f:
    pickle.dump({'mu_arr': mu_arr, 'solutions': solutions}, f)

# Vary $\epsilon$ and $\mu$ (3D bifurcation diagram)

In [ ]:
rng = np.random.default_rng(42)  # For reproducibility

# Parameters
steps = 1000
N = 200

N_mu = 100
N_eps = 100
mu_arr = np.linspace(-0.1, 1.0, N_mu)
epsilon_arr = 10**np.linspace(-3, -1, N_eps)
mu2, eps2 = np.meshgrid(mu_arr, epsilon_arr)
mu_arr = mu2.flatten()
epsilon_arr = eps2.flatten()

solutions = np.empty((len(mu_arr), steps+1, N), dtype=float)

for i, (mu, eps) in enumerate(zip(mu_arr, epsilon_arr)):
    print(f'Solving for mu = {mu}, epsilon = {eps}')

    # Solve the Allen-Cahn equation
    sol = solve_AllenCahn(rng=rng, mu=mu, epsilon=eps, steps=steps, dt=0.1, N=N, boundary_condition='periodic')

    solutions[i] = sol

In [ ]:
solutions.shape

In [ ]:
import pickle
with open('AllenCahn_periodic_varyEpsMu_onlyFinal.pkl', 'wb') as f:
    pickle.dump({'mu_arr': mu_arr, 'epsilon_arr': epsilon_arr, 'solutions': solutions[:, -1]}, f)

In [ ]:
with open('AllenCahn_periodic_varyEpsMu_1.pkl', 'wb') as f:
    pickle.dump({'mu_arr': mu_arr, 'epsilon_arr': epsilon_arr, 'solutions': solutions[:5000]}, f)

In [ ]:
with open('AllenCahn_periodic_varyEpsMu_2.pkl', 'wb') as f:
    pickle.dump({'mu_arr': mu_arr, 'epsilon_arr': epsilon_arr, 'solutions': solutions[5000:]}, f)

In [ ]:
%matplotlib qt
# Solution norm 3D plot
fig, ax = plt.subplots(figsize=(8, 6), dpi=200, subplot_kw={"projection": "3d"})
solution_norms = np.linalg.norm(solutions[:, -1], axis=(-1))
ax.scatter(mu_arr, epsilon_arr, solution_norms, s=3, c=solution_norms, cmap='viridis')
ax.set_xlabel(r'$\mu$')
ax.set_ylabel(r'$\epsilon$')
ax.set_zlabel(r'Solution norm')

# Steady-state

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.optimize import root

# Parameters
N = 200  # number of grid points
L = 1.0  # domain length
x = np.linspace(0, L, N)
dx = x[1] - x[0]

# Finite difference Laplacian operator with Dirichlet boundary conditions
diagonals = [np.ones(N-1), -2*np.ones(N), np.ones(N-1)]
laplacian = diags(diagonals, offsets=[-1, 0, 1], shape=(N, N)).toarray()
laplacian[0, 1] = 2.0   # Neumann BC at left boundary
laplacian[-1, -2] = 2.0 # Neumann BC at right boundary
laplacian = laplacian / dx**2

# Allen-Cahn equation steady-state residual
def residual(u, epsilon):
    return epsilon**2 * laplacian.dot(u) - (u**3 - u)

# Homotopy continuation
epsilons = np.linspace(0.01, 0.3, 100)
solutions = []

rng = np.random.default_rng(42)

# Initial guesses
u0 = [np.zeros(N),
      np.ones(N),
      -np.ones(N)] + [
          -1 + 2/19*i + rng.normal(size=N)/10 for i in range(20)
      ]

for eps in epsilons:
    # print(f'\n\nSolving for epsilon = {eps}')
    # print('Initial guesses:\n', u0, '\n')
    sols_temp = []
    for u0_temp in u0:
        # print('u0_temp', u0_temp)
        # print('residual(u0_temp)', residual(u0_temp, eps))
        # print('eps', eps)
        sol = root(residual, u0_temp, args=(eps,), method='hybr')
        if sol.success:
            u0_temp = sol.x  # update initial guess for next epsilon
            sols_temp.append((eps, u0_temp))
    solutions.extend(sols_temp)

    u0 = [sol[1] for sol in sols_temp]
    u0.extend([rng.normal(size=N) for _ in range(23-len(u0))])

# Plot bifurcation diagram: max(u) vs epsilon
eps_vals = [eps for eps, u in solutions if u is not None]

## Plot max|u| vs $\epsilon$

In [ ]:
u_max_vals = [np.max(np.abs(u)) for eps, u in solutions if u is not None]
plt.figure(figsize=(8, 6))
plt.scatter(eps_vals, u_max_vals, s=1)
plt.xlabel('ε')
plt.ylabel('max|u|')
plt.title('Bifurcation Diagram of 1D Allen-Cahn Equation')
plt.grid(True)
plt.show()

## Plot u[0] vs $\epsilon$

In [ ]:
u_boundary_vals = [u[0] for eps, u in solutions if u is not None]
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(eps_vals, u_boundary_vals, s=1, alpha=0.3)
plt.xlabel('ε')
plt.ylabel('u(0)')
plt.title('1D Allen-Cahn bifurcation diagram, steady-state')
plt.grid(True)
plt.show()

## Plot all solutions

In [ ]:
for sol in solutions:
    plt.plot(sol[1])